In [1]:
import os
import select
import sys
import threading
import queue
__name__ = "__jpnotebook__"

In [2]:
from time import sleep
def func():
    i = 1
    while i < 11:
        print("client %d"%i, flush=True)
        sleep(3)
        i += 1
if __name__ == "__main__":
    func()

In [3]:
fn = "nonblock_test.py"
with open(fn, 'w') as f:
    f.write(In[-2])

In [4]:
import subprocess
mk_popen = lambda: subprocess.Popen("python %s"%fn, 0, shell=False, stdout=subprocess.PIPE)

In [9]:
class AsyncPipeReader(threading.Thread):
    def __init__(self, fp, q):
        super().__init__(None, daemon=True)
        self.fp = fp
        self.q = q
        self.running = False
        self.eof = False
        
    def run(self):
        self.running = True
        while self.running:
            try:
                c = self.fp.read(1)
            except Exception:
                break
            if c == b"":
                self.eof = True
                break
            else:
                self.q.put(c)
     
def _read_all(q):
    b = []
    with q.mutex:
        while q.queue:
            b.append(q.queue.popleft())
    return b
    
def _read_async(reader, q):
    readall = _read_all
    while True:
        b = readall(q)
        yield b"".join(b)
        if reader.eof:
            break
    reader.running = False
    yield
            
def read_async(p):
    global reader
    q = queue.Queue()
    reader = AsyncPipeReader(p.stdout, q)
    reader.start()
    gen = _read_async(reader, q)
    gen.reader = reader
    gen.q = q
    next(gen)
    return gen.__next__

In [10]:
read = read_async(mk_popen())
print("beginning")
i = 0
while True:
    i += 1
    c = read()
    if c:
        sys.stdout.write(c)
        sys.stdout.flush()
    elif c == b"":
        print("\ri=%d"%i, end="")
    else:
        break
    sleep(0.1)

AttributeError: 'generator' object has no attribute 'reader'

In [11]:
queue??